In [2]:
import os
import cv2
import glob
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns

from keras.utils import np_utils 
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler,ModelCheckpoint
from keras.metrics import categorical_accuracy, categorical_crossentropy
from keras.layers import Input, Convolution2D, MaxPooling2D, Flatten, Dense, Dropout

warnings.filterwarnings("ignore")

In [3]:
USE_DROPOUT = False
LEARN_RATE = 0.001
Height = 256
Weight = 256
Channel= 3

In [25]:
def get_train_holdout_files(current_iteration = 1):
    
    print("Get train/holdout files.")
        
    src_dir = "D:/jupyter-notebook/LiverCancer/Data_Description/" + str(current_iteration) +"/"

    #分割训练数据和测试数据  
    train_samples = pd.read_csv(src_dir + "train.csv")["file_path"].tolist()
    holdout_samples = pd.read_csv(src_dir + "holdout.csv")["file_path"].tolist()
    print("Train Count: ", len(train_samples), ", Holdout Count: ", len(holdout_samples))

    #建立描述集合
    train_rep = []
    holdout_rep = []
    sets = [[train_rep, train_samples], [holdout_rep, holdout_samples]]

    for set_item in sets:

        rep = set_item[0]
        samples = set_item[1]

        for index, sample_path in enumerate(samples):

            if "grade0" in sample_path:
                sample_label = 0
            elif "grade1" in sample_path:
                sample_label = 1
            elif "grade2" in sample_path:
                sample_label = 2
            elif "grade3" in sample_path:
                sample_label = 3
            elif "grade4" in sample_path:
                sample_label = 4

            rep.append([sample_path, sample_label])

    print("Train Count: ", len(train_rep), ", Holdout Count: ", len(holdout_rep))

    return train_rep, holdout_rep

In [26]:
def data_generator(batch_size, record_list, train_set):
    
    while True:
        
        batch_index = 0
        image_list = []
        label_list = []

        if train_set:
            random.shuffle(record_list)

        #逐一遍历所有数据
        for index, record_item in enumerate(record_list):

            sample_path = record_item[0]
            sample_label = record_item[1]

            #转换成多分类标签
            sample_label = np_utils.to_categorical(sample_label,5)  

            #读取图片、修改尺寸、标准化
            sample_image = cv2.imread(sample_path)
            sample_image = (sample_image - np.average(sample_image)) / np.std(sample_image)
            sample_image = sample_image.reshape(1, sample_image.shape[0], sample_image.shape[1], 3)

            #数据增强
            if train_set:   
                if random.randint(0, 100) > 50:
                    sample_image = np.fliplr(sample_image)
                if random.randint(0, 100) > 50:
                    sample_image = np.flipud(sample_image)
                if random.randint(0, 100) > 50:
                    sample_image = sample_image[:,::-1]
                if random.randint(0, 100) > 50:
                    sample_image = sample_image[::-1, :]

            #添加数据
            image_list.append(sample_image)
            label_list.append(sample_label)
            batch_index += 1

            if batch_index >= batch_size:
                x = np.vstack(image_list)
                y = np.vstack(label_list)
                yield x, y
                image_list = []
                label_list = []
                batch_index = 0

In [28]:
train_files, holdout_files = get_train_holdout_files(current_iteration = 1)

Get train/holdout files.
Train Count:  266 , Holdout Count:  30
Train Count:  266 , Holdout Count:  30


In [29]:
def get_net(input_shape=(Height, Weight, Channel), load_weight_path=None) -> Model:  #期待返回类型为model
    
    inputs = Input(shape=input_shape, name="input")
    x = inputs
    x = Convolution2D(32, 3, 3, activation='relu', border_mode='same', name='conv1a', subsample=(1, 1))(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool1')(x)
    if USE_DROPOUT:
        x = Dropout(p=0.3)(x)

    # 2nd layer group
    x = Convolution2D(32, 3, 3, activation='relu', border_mode='same', name='conv2a', subsample=(1, 1))(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool2')(x)
    if USE_DROPOUT:
        x = Dropout(p=0.3)(x)

    # 3rd layer group
    x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='conv3b', subsample=(1, 1))(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool3')(x)
    if USE_DROPOUT:
        x = Dropout(p=0.3)(x)

    # 4th layer group
    x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='conv4b', subsample=(1, 1),)(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool4')(x)
    if USE_DROPOUT:
        x = Dropout(p=0.3)(x)
        
    # 5th layer group
    x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='conv5a', subsample=(1, 1))(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool5')(x)
    if USE_DROPOUT:
        x = Dropout(p=0.3)(x)
        
    # 6th layer group
    x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='conv6a', subsample=(1, 1))(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool6')(x)
    if USE_DROPOUT:
        x = Dropout(p=0.3)(x)
        
    # 7th layer group
    x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='conv7a', subsample=(1, 1))(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool7')(x)
    if USE_DROPOUT:
        x = Dropout(p=0.3)(x)
        
    # 8th layer group
    x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='conv8a', subsample=(1, 1))(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool8')(x)
    if USE_DROPOUT:
        x = Dropout(p=0.3)(x)
    
    #输出
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    out_class = Dense(5, activation='softmax', name='out_class')(x)

    model = Model(input=inputs, output=out_class)
    
    if load_weight_path is not None:
        model.load_weights(load_weight_path, by_name=False)

    #编译模型
    model.compile(optimizer=SGD(lr=LEARN_RATE, momentum=0.9, nesterov=True), loss={ "out_class": "categorical_crossentropy" }, metrics={"out_class": [categorical_accuracy, categorical_crossentropy] } )
    model.summary(line_length=140)

    return model

In [30]:
get_net()

C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name="conv1a", strides=(1, 1), padding="same")`
  """
C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="pool1", padding="valid")`
  
C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name="conv2a", strides=(1, 1), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="pool2", padding="va

____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input (InputLayer)                                             (None, 256, 256, 3)                                     0                    
____________________________________________________________________________________________________________________________________________
conv1a (Conv2D)                                                (None, 256, 256, 32)                                    896                  
____________________________________________________________________________________________________________________________________________
pool1 (MaxPooling2D)                                           (None, 128, 128, 32)                                    0                    
_____________

C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ou...)`


In [34]:
model_name = "liver_CNN"
load_weights_path=None
batch_size = 8

for i in range(10):

    model_name_i =model_name + "_" + str(i+1)
    
    #获得训练和测试集合，以：路径、class label的形式保存
    train_files, holdout_files = get_train_holdout_files( current_iteration = (i+1) )

    #训练数据集
    train_gen = data_generator(batch_size, train_files, train_set=True)

    #测试数据集
    holdout_gen = data_generator(batch_size, holdout_files, train_set=False)

    #动态设置学习率
    learnrate_scheduler = LearningRateScheduler(step_decay)

    #获取model
    model = get_net(load_weight_path=load_weights_path)

    checkpoint = ModelCheckpoint("workdir/model_" + model_name_i + "_"  + "_e" + "{epoch:02d}-{val_loss:.4f}.hd5", monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    checkpoint_best = ModelCheckpoint("workdir/model_" + model_name_i + "_"  + "_best.hd5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    
    model.fit_generator(generator=train_gen, samples_per_epoch=len(train_files), nb_epoch=10, verbose=1, validation_data=holdout_gen, nb_val_samples=len(holdout_files), class_weight="auto", callbacks=[checkpoint, checkpoint_best, learnrate_scheduler])

    model.save("workdir/model_" + model_name_i + "_end.hd5")

Get train/holdout files.
Train Count:  266 , Holdout Count:  30
Train Count:  266 , Holdout Count:  30


C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name="conv1a", strides=(1, 1), padding="same")`
  """
C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="pool1", padding="valid")`
  
C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name="conv2a", strides=(1, 1), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="pool2", padding="va

____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input (InputLayer)                                             (None, 256, 256, 3)                                     0                    
____________________________________________________________________________________________________________________________________________
conv1a (Conv2D)                                                (None, 256, 256, 32)                                    896                  
____________________________________________________________________________________________________________________________________________
pool1 (MaxPooling2D)                                           (None, 128, 128, 32)                                    0                    
_____________


Epoch 00006: saving model to workdir/model_liver_CNN_1__e06-0.1296.hd5
Epoch 7/10
learnrate:  0.001  epoch:  6
266/266 [==============================] - 38s 144ms/step - loss: 0.1886 - categorical_accuracy: 0.9455 - categorical_crossentropy: 0.1886 - val_loss: 0.0610 - val_categorical_accuracy: 0.9583 - val_categorical_crossentropy: 0.0610

Epoch 00007: saving model to workdir/model_liver_CNN_1__e07-0.0610.hd5
Epoch 8/10
learnrate:  0.001  epoch:  7
266/266 [==============================] - 39s 146ms/step - loss: 0.1034 - categorical_accuracy: 0.9680 - categorical_crossentropy: 0.1034 - val_loss: 0.0717 - val_categorical_accuracy: 0.9583 - val_categorical_crossentropy: 0.0717

Epoch 00008: saving model to workdir/model_liver_CNN_1__e08-0.0717.hd5
Epoch 9/10
learnrate:  0.001  epoch:  8
266/266 [==============================] - 40s 149ms/step - loss: 0.0434 - categorical_accuracy: 0.9868 - categorical_crossentropy: 0.0434 - val_loss: 0.0198 - val_categorical_accuracy: 1.0000 - val_c

266/266 [==============================] - 38s 144ms/step - loss: 0.9119 - categorical_accuracy: 0.5625 - categorical_crossentropy: 0.9119 - val_loss: 0.6614 - val_categorical_accuracy: 0.6417 - val_categorical_crossentropy: 0.6614

Epoch 00002: saving model to workdir/model_liver_CNN_2__e02-0.6614.hd5
Epoch 3/10
learnrate:  0.001  epoch:  2
266/266 [==============================] - 39s 146ms/step - loss: 0.7077 - categorical_accuracy: 0.6725 - categorical_crossentropy: 0.7077 - val_loss: 1.1117 - val_categorical_accuracy: 0.5625 - val_categorical_crossentropy: 1.1117

Epoch 00003: saving model to workdir/model_liver_CNN_2__e03-1.1117.hd5
Epoch 4/10
learnrate:  0.001  epoch:  3
266/266 [==============================] - 39s 145ms/step - loss: 0.6079 - categorical_accuracy: 0.7091 - categorical_crossentropy: 0.6079 - val_loss: 0.7944 - val_categorical_accuracy: 0.7500 - val_categorical_crossentropy: 0.7944

Epoch 00004: saving model to workdir/model_liver_CNN_2__e04-0.7944.hd5
Epoch 5/

...

C:\Users\kingbook\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., verbose=1, validation_data=<generator..., class_weight="auto", callbacks=[<keras.ca..., steps_per_epoch=267, epochs=10, validation_steps=29)`


Epoch 1/10
learnrate:  0.001  epoch:  0
267/267 [==============================] - 41s 152ms/step - loss: 1.3788 - categorical_accuracy: 0.3886 - categorical_crossentropy: 1.3788 - val_loss: 1.0659 - val_categorical_accuracy: 0.5000 - val_categorical_crossentropy: 1.0659

Epoch 00001: saving model to workdir/model_liver_CNN_7__e01-1.0659.hd5
Epoch 2/10
learnrate:  0.001  epoch:  1
267/267 [==============================] - 38s 143ms/step - loss: 0.8147 - categorical_accuracy: 0.6175 - categorical_crossentropy: 0.8147 - val_loss: 0.8626 - val_categorical_accuracy: 0.7241 - val_categorical_crossentropy: 0.8626

Epoch 00002: saving model to workdir/model_liver_CNN_7__e02-0.8626.hd5
Epoch 3/10
learnrate:  0.001  epoch:  2
267/267 [==============================] - 38s 144ms/step - loss: 0.6546 - categorical_accuracy: 0.7065 - categorical_crossentropy: 0.6546 - val_loss: 0.7519 - val_categorical_accuracy: 0.5388 - val_categorical_crossentropy: 0.7519

Epoch 00003: saving model to workdir/mo

Epoch 1/10
learnrate:  0.001  epoch:  0
267/267 [==============================] - 41s 153ms/step - loss: 1.4098 - categorical_accuracy: 0.3890 - categorical_crossentropy: 1.4098 - val_loss: 0.9973 - val_categorical_accuracy: 0.4612 - val_categorical_crossentropy: 0.9973

Epoch 00001: saving model to workdir/model_liver_CNN_8__e01-0.9973.hd5
Epoch 2/10
learnrate:  0.001  epoch:  1
267/267 [==============================] - 38s 143ms/step - loss: 0.8667 - categorical_accuracy: 0.5974 - categorical_crossentropy: 0.8667 - val_loss: 1.1083 - val_categorical_accuracy: 0.4569 - val_categorical_crossentropy: 1.1083

Epoch 00002: saving model to workdir/model_liver_CNN_8__e02-1.1083.hd5
Epoch 3/10
learnrate:  0.001  epoch:  2
267/267 [==============================] - 38s 143ms/step - loss: 0.7343 - categorical_accuracy: 0.6667 - categorical_crossentropy: 0.7343 - val_loss: 0.7417 - val_categorical_accuracy: 0.6034 - val_categorical_crossentropy: 0.7417

Epoch 00003: saving model to workdir/mo

Epoch 1/10
learnrate:  0.001  epoch:  0
267/267 [==============================] - 40s 151ms/step - loss: 1.3555 - categorical_accuracy: 0.3928 - categorical_crossentropy: 1.3555 - val_loss: 0.7228 - val_categorical_accuracy: 0.6552 - val_categorical_crossentropy: 0.7228

Epoch 00001: saving model to workdir/model_liver_CNN_9__e01-0.7228.hd5
Epoch 2/10
learnrate:  0.001  epoch:  1
267/267 [==============================] - 38s 141ms/step - loss: 0.8461 - categorical_accuracy: 0.6044 - categorical_crossentropy: 0.8461 - val_loss: 0.6535 - val_categorical_accuracy: 0.6853 - val_categorical_crossentropy: 0.6535

Epoch 00002: saving model to workdir/model_liver_CNN_9__e02-0.6535.hd5
Epoch 3/10
learnrate:  0.001  epoch:  2
267/267 [==============================] - 38s 142ms/step - loss: 0.6254 - categorical_accuracy: 0.7051 - categorical_crossentropy: 0.6254 - val_loss: 0.2805 - val_categorical_accuracy: 0.8707 - val_categorical_crossentropy: 0.2805

Epoch 00003: saving model to workdir/mo

Epoch 1/10
learnrate:  0.001  epoch:  0
267/267 [==============================] - 44s 164ms/step - loss: 1.4952 - categorical_accuracy: 0.3221 - categorical_crossentropy: 1.4952 - val_loss: 1.4913 - val_categorical_accuracy: 0.2586 - val_categorical_crossentropy: 1.4913

Epoch 00001: saving model to workdir/model_liver_CNN_10__e01-1.4913.hd5
Epoch 2/10
learnrate:  0.001  epoch:  1
267/267 [==============================] - 44s 166ms/step - loss: 0.8967 - categorical_accuracy: 0.5880 - categorical_crossentropy: 0.8967 - val_loss: 0.7081 - val_categorical_accuracy: 0.6897 - val_categorical_crossentropy: 0.7081

Epoch 00002: saving model to workdir/model_liver_CNN_10__e02-0.7081.hd5
Epoch 3/10
learnrate:  0.001  epoch:  2
267/267 [==============================] - 40s 149ms/step - loss: 0.7121 - categorical_accuracy: 0.6643 - categorical_crossentropy: 0.7121 - val_loss: 0.6402 - val_categorical_accuracy: 0.7802 - val_categorical_crossentropy: 0.6402

Epoch 00003: saving model to workdir/

In [31]:
def step_decay(epoch):
    res = 0.001
    if epoch > 100:
        res = 0.0001
    print("learnrate: ", res, " epoch: ", epoch)
    return res